In [2]:
from scipy.optimize import linprog
import numpy as np
import random

## Task 2

In [3]:
# Compute the best strategy to schedule the use of the appliances
off_peak_hours = range(0-16) and range(21-24)
peak_hours = range(17-20)
lighting_hours = range(10-20) # switch appliance?

# Non-shiftable appliances
lighting = 1.5
refrigerator = 2
tv = 0.6
electric_stove = 3.9

# Shiftable appliances
dishwasher = 1.44
laundry_machine = 1.94
cloth_dryer = 2.50
electric_vehicle = 9.9

In [4]:
# using a random function to generate the pricing curve in a day
def generate_pricing_curve():
    random.seed(6)
    price_curve = []
    for i in range(24):
        if i in peak_hours:
            # higher price in the peak hours
            price_curve.append(random.uniform(1, 1.5))
        else:
            # lower price in the off-peak hours
            price_curve.append(random.uniform(0.5, 1))
    return price_curve

pricing_curve = generate_pricing_curve()
print(pricing_curve)

[0.8966700418808315, 0.9109770211598633, 0.7425173139654726, 0.630810741472329, 0.5002258574425356, 0.8314092814418839, 0.7351271285322225, 0.8798653175489466, 0.6865801860369207, 0.8850699179689951, 0.6363490428359854, 0.9009577415813018, 0.864912416311008, 0.7070032205826517, 0.7691526097776384, 0.8410258706443392, 0.5964924378820438, 0.7768075827491421, 0.9025620249244866, 0.632760527219251, 0.9016826548056567, 0.8428449410502205, 0.9221411623980824, 0.6677910089129071]


In [5]:
# write a program in order to minimize energy cost
# pricing_curve
# linprog


In [6]:
# matplot for flow chart


## Task 3

In [7]:
# Requirement: Small neighborhood that has 30 households
households = 30

# Each household has the same setting as that in question 2


# Only a fraction of the households owns an EV. 
household_withEV = 10

# Real-Time Pricing (RTP) scheme
pricing_curve = generate_pricing_curve()

# Compute the best strategy for scheduling the appliances and write a program in order to minimize energy cost in the neighborhood. 
# for(len(range(3)):


# write a program in order to minimize energy cost
# pricing_curve
# linprog

SyntaxError: invalid syntax (697549062.py, line 14)

## Task 4

In [8]:
"""
We may observe that in some timeslots, energy consumption is very high. 
Then, the objective function should consider both energy cost and the peak load. 

if (total energy consumption of all appliances) > peak load:
    add_constraints

"""

peak_load = 0

In [9]:
# Please use the same setting of the appliances

# Real-Time Pricing (RTP) scheme
pricing_curve = generate_pricing_curve()

# Please formulate an optimization problem and compute the best strategy to schedule the use of the appliances.